In [29]:
from selenium.webdriver.support.wait import WebDriverWait
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

In [30]:
review_id = 0

def crawl_data(url, res_id):
    global review_id
    browser = webdriver.Chrome()
    browser.get(url)

    results = []

    # Chờ phần review hiển thị
    try:
        WebDriverWait(browser, 15).until(
            EC.presence_of_element_located((By.CLASS_NAME, "review-item"))
        )
    except:
        print(f"⛔ Không tìm thấy review nào cho {url}")
        browser.quit()
        return results

    # Nhấn "Xem thêm bình luận" cho đến khi hết
    while True:
        try:
            load_more = WebDriverWait(browser, 5).until(
                EC.element_to_be_clickable(
                    (By.XPATH, '//a[@class="fd-btn-more"]/label[contains(text(),"Xem thêm bình luận")]'))
            )
            browser.execute_script("arguments[0].click();", load_more)
            WebDriverWait(browser, 5).until(  # Wait for new content to load
                lambda driver: len(driver.find_elements(By.CLASS_NAME, "review-item")) > len(results)
            )
        except Exception:
            break

    comments = browser.find_elements(By.CLASS_NAME, "review-item")

    # Xuất dữ liệu review
    for c in comments:
        try:
            username = c.find_element(By.CSS_SELECTOR, ".ru-username").text.strip()
            review_text = c.find_element(By.CSS_SELECTOR, ".rd-des span").text.strip()
            score_text = c.find_element(By.CSS_SELECTOR, ".review-points span").text.strip()

        except:
            username = "Người dùng ẩn danh"  # hoặc "Ẩn danh"
            review_text = "Đã bình luận ẩn danh"
            score_text = "0.0"

        review = {
            "review_id": review_id,
            "username": username,
            "review_text": review_text,
            "score_text": score_text,
            "res_id": res_id,
        }
        review_id = review_id + 1
        results.append(review)
    browser.close()

    return results


In [31]:
urls = [
    'https://www.foody.vn/ho-chi-minh/mi-dau-hao-hongkong-thuong-hang-%E6%B8%AF%E5%BC%8F%E8%9A%9D%E6%B2%B9%E6%8D%9E%E9%9D%A2-chachaanteng-phan-phu-tien',
    'https://www.foody.vn/ho-chi-minh/dau-hu-ca-vien-hongkong-%E6%B8%AF%E5%BC%8F%E9%B1%BC%E4%B8%B8%E8%B1%86%E8%85%90-chachaanteng',
    'https://www.foody.vn/ho-chi-minh/kaya-toast-nuong-ca-vien-%E5%92%96%E6%A4%B0%E5%90%90%E5%8F%B8-%E9%B1%BC%E4%B8%B8-chachaanteng',
    'https://www.foody.vn/ho-chi-minh/american-hotdog',
    'https://www.foody.vn/ho-chi-minh/%E8%8C%B6%E9%A4%90%E5%8E%85-cha-chaan-teng-am-thuc-hongkong',
    'https://www.foody.vn/ho-chi-minh/%E6%B8%AF%E5%BC%8F%E7%94%9C%E5%93%81%E5%BA%97-troi-nuoc-hongkong-chachaanteng',
    'https://www.foody.vn/ho-chi-minh/%E6%B8%AF%E5%BC%8F%E7%82%B9%E5%BF%83%E7%81%AB%E9%94%85-dimsum-lau-hai-san-hongkong',
    'https://www.foody.vn/ho-chi-minh/%E5%90%90%E5%8F%B8%E5%B7%A5%E5%9D%8A-%E5%A5%B6%E8%8C%B6%E6%A1%A3-phan-phu-tien',
    'https://www.foody.vn/ho-chi-minh/che-hongkong-%E6%B8%AF%E5%BC%8F%E7%B3%96%E6%B0%B4-phan-phu-tien',
    'https://www.foody.vn/ho-chi-minh/xoi-cade-tran-phu-chinh-goc-%E5%92%96%E6%A4%B0%E7%B3%AF%E7%B1%B3%E9%A5%AD',
    'https://www.foody.vn/ho-chi-minh/lau-ca-thac-lac-tuoi-dac-biet-phan-phu-tien.jj93kq',
    'https://www.foody.vn/ho-chi-minh/steak-zone',
    'https://www.foody.vn/ho-chi-minh/truong-thanh-com-ga-xoi-mo-le-thi-hong',
    'https://www.foody.vn/ho-chi-minh/pha-lau-rubi',
    'https://www.foody.vn/ho-chi-minh/banh-mi-t-phan-dang-luu',
    'https://www.foody.vn/ho-chi-minh/sui-cao-thien-thien-ha-ton-quyen',
    'https://www.foody.vn/ho-chi-minh/mi-kho-xa-xiu-gia-phuc',
    'https://www.foody.vn/ho-chi-minh/pizza-hut-thong-nhat',
    'https://www.foody.vn/ho-chi-minh/oc-cam-tan-my',
    'https://www.foody.vn/ho-chi-minh/domino-s-pizza-nguyen-duy-trinh',
    'https://www.foody.vn/ho-chi-minh/che-thai-ut-lyly',
    'https://www.foody.vn/ho-chi-minh/banh-canh-cua-260-tan-son-nhi',
    'https://www.foody.vn/ho-chi-minh/banh-trang-toi-019',
    'https://www.foody.vn/ho-chi-minh/tra-sua-maycha-dong-den',
    'https://www.foody.vn/ho-chi-minh/hoa-co-com-van-phong-nguyen-van-sang',
    'https://www.foody.vn/ho-chi-minh/banh-cuon-ba-mien',
    'https://www.foody.vn/ho-chi-minh/bun-quay-kien-xay-dien-bien-phu',
    'https://www.foody.vn/ho-chi-minh/gio-cha-minh-chau',
    'https://www.foody.vn/ho-chi-minh/3a-korean-food-ga-sot-pho-mai-mi-cay-han-quoc-tokbokki-nguyen-van-luong',
    'https://www.foody.vn/ho-chi-minh/com-tam-tai-476-nguyen-tri-phuong',
    'https://www.foody.vn/ho-chi-minh/mr-nghiem-saigon-chao-ech-chao-cua-singapore-nguyen-huu-tho',
    'https://www.foody.vn/ho-chi-minh/anh-cuong-bakery-banh-kem-banh-mi-nguyen-van-lac',
    'https://www.foody.vn/ho-chi-minh/tau-hu-coc-hoa-su',
    'https://www.foody.vn/ho-chi-minh/tra-va-sua',
    'https://www.foody.vn/ho-chi-minh/co-thuy-mi-xao-gion',
    'https://www.foody.vn/ho-chi-minh/chao-trang-hot-vit-muoi-tran-khac-chan',
    'https://www.foody.vn/ho-chi-minh/sui-cao-dai-nuong',
    'https://www.foody.vn/ho-chi-minh/hien-banh-canh-cua',
    'https://www.foody.vn/ho-chi-minh/tu-thuy-mi-hen',
    'https://www.foody.vn/ho-chi-minh/bun-dau-mac-van-khoa-ten-lua',
    'https://www.foody.vn/ho-chi-minh/com-nieu-phuong-nam-com-van-phong',
    'https://www.foody.vn/ho-chi-minh/lotteria-ly-thai-to',
    'https://www.foody.vn/ho-chi-minh/tra-sua-kem-tuyet-hong-phuc-togo',
    'https://www.foody.vn/ho-chi-minh/kfc-xo-viet-nghe-tinh',
    'https://www.foody.vn/ho-chi-minh/phuc-long-phan-van-tri',
    'https://www.foody.vn/ho-chi-minh/baoz-dimsum-resraurant',
    'https://www.foody.vn/ho-chi-minh/chi-bo-quan-bun-bo',
    'https://www.foody.vn/ho-chi-minh/starbucks-coffee-lotte-nsg',
    'https://www.foody.vn/ho-chi-minh/com-chay-phuoc-hoa',
    'https://www.foody.vn/ho-chi-minh/ca-nuong-mien-trung',
    'https://www.foody.vn/ho-chi-minh/me-vui-29-tran-quoc-toan',
    'https://www.foody.vn/ho-chi-minh/banh-mi-to-bo-kho-xiu-mai-lagu-ga-duong-d5',
    'https://www.foody.vn/ho-chi-minh/com-ga-hai-nam-hau-giang',
    'https://www.foody.vn/ho-chi-minh/chi-ut-vu-heo-nuong-minh-phung',
    'https://www.foody.vn/ho-chi-minh/phuc-long-ly-thuong-kiet',
    'https://www.foody.vn/ho-chi-minh/seva-nha-hang-chay',
    'https://www.foody.vn/ho-chi-minh/tra-sua-tocotoco-su-van-hanh',
    'https://www.foody.vn/ho-chi-minh/kim-huong-bun-bo-hue-nguyen-xi',
    'https://www.foody.vn/ho-chi-minh/com-tam-06',
    'https://www.foody.vn/ho-chi-minh/co-ba-che-com-tam-long-xuyen-nguyen-xi',
    'https://www.foody.vn/ho-chi-minh/che-via-he-vo-van-tan',
    'https://www.foody.vn/ho-chi-minh/banh-xeo-335',
    'https://www.foody.vn/ho-chi-minh/lau-ca-keo-soc-trang',
    'https://www.foody.vn/ho-chi-minh/ga-ta-que-nha',
    'https://www.foody.vn/ho-chi-minh/milo-dam-chung-cu-42',
    'https://www.foody.vn/ho-chi-minh/tiramisu-kieu-italy-nguyen-ban-thu-cong-breakfast-shop-phan-phu-tien',
    'https://www.foody.vn/ho-chi-minh/che-nguoi-hoa-an-diem',
    'https://www.foody.vn/ho-chi-minh/quan-com-thin-ky',
    'https://www.foody.vn/ho-chi-minh/hong-tra-ngo-gia-cu-xa-phu-lam-b',
    'https://www.foody.vn/ho-chi-minh/tu-anh-com-ga-xoi-mo',
    'https://www.foody.vn/ho-chi-minh/bun-rieu-banh-da-cua-cha-ngo-thi-thu-minh',
    'https://www.foody.vn/ho-chi-minh/bun-oc-rieu-cua-thanh-hai-ky-dong',
    'https://www.foody.vn/ho-chi-minh/com-tam-449-an-duong-vuong',
    'https://www.foody.vn/ho-chi-minh/ha-cao-ngoc-lan',
    'https://www.foody.vn/ho-chi-minh/hai-ky-mi-gia-nguyen-trai',
    'https://www.foody.vn/ho-chi-minh/din-chon-mi-gia',
    'https://www.foody.vn/ho-chi-minh/lau-bo-co-thao-ho-thi-ky',
    'https://www.foody.vn/ho-chi-minh/vinh-an-chay-com-suon-bi-cha-chay-bun-thit-nuong-cha-gio-chay-au-co',
    'https://www.foody.vn/ho-chi-minh/domino-s-pizza-nguyen-tri-phuong',
    'https://www.foody.vn/ho-chi-minh/tuktuk-thai-bistro-ly-tu-trong',
    'https://www.foody.vn/ho-chi-minh/tri-phuong-lau-bo',
    'https://www.foody.vn/ho-chi-minh/mutsumian-luon-nuong-than',
    'https://www.foody.vn/ho-chi-minh/a-ma-kitchen-mon-hongkong',
    'https://www.foody.vn/ho-chi-minh/tau-hu-an-chi',
    'https://www.foody.vn/ho-chi-minh/kim-huong-bun-bo-hue-hoang-hoa-tham',
    'https://www.foody.vn/ho-chi-minh/com-tam-co-ba-sai-gon-le-quang-dinh',
    'https://www.foody.vn/ho-chi-minh/xoi-xoi-xoi-xien-nuong',
    'https://www.foody.vn/ho-chi-minh/kitachi-mi-cay-han-quoc-7-cap-do-155-van-kiep',
    'https://www.foody.vn/ho-chi-minh/quan-thao-hien-dac-san-mien-trung',
    'https://www.foody.vn/ho-chi-minh/sakura-udon-japanese-fast-food',
    'https://www.foody.vn/ho-chi-minh/hai-cay-dua',
    'https://www.foody.vn/ho-chi-minh/banh-mi-hoang-phat',
    'https://www.foody.vn/ho-chi-minh/tien-hai-quan-bun-dau-mam-tom-ha-noi',
    'https://www.foody.vn/ho-chi-minh/tiem-nha-ngan-tra-trai-cay-an-vat',
    'https://www.foody.vn/ho-chi-minh/com-ga-singapore-quan-3-214c-ly-chinh-thang',
    'https://www.foody.vn/ho-chi-minh/tuyet-nhung-com-ga-phu-yen',
    'https://www.foody.vn/ho-chi-minh/highlands-coffee-tra-ca-phe-banh-402-tran-hung-dao-b',
    'https://www.foody.vn/ho-chi-minh/busan-korean-food-mon-han-quoc-dong-nai',
    'https://www.foody.vn/ho-chi-minh/com-tam-phuc-loc-tho-phan-van-tri',
    'https://www.foody.vn/ho-chi-minh/hu-tieu-co-ba-1588-huynh-tan-phat',
    'https://www.foody.vn/ho-chi-minh/banh-mi-mat-troi-moc-huong-vi-tho-nhi-ky',
    'https://www.foody.vn/ho-chi-minh/cua-hang-banh-canh-bot-gao-gio-heo-23a-phu-my',
    'https://www.foody.vn/ho-chi-minh/quan-688-hu-tieu-mi-suon',
    'https://www.foody.vn/ho-chi-minh/la-garden-restaurant',
    'https://www.foody.vn/ho-chi-minh/banh-mi-sau-le-nguyen-tri-phuong',
    'https://www.foody.vn/ho-chi-minh/banh-mi-hong-phat-nguyen-tri-phuong',
    'https://www.foody.vn/ho-chi-minh/tiem-banh-mi-nhung-chang-trai-thanh-thai',
    'https://www.foody.vn/ho-chi-minh/bun-dau-mam-tom-xoai',
    'https://www.foody.vn/ho-chi-minh/banh-bao-hu-tieu-ca-can',
    'https://www.foody.vn/ho-chi-minh/quan-chay-bat-nha-1',
    'https://www.foody.vn/ho-chi-minh/com-chao-trieu-chau',
    'https://www.foody.vn/ho-chi-minh/com-tam-huyen-dinh-tien-hoang',
    'https://www.foody.vn/ho-chi-minh/hai-con-bo-nguyen-thi-minh-khai',
    'https://www.foody.vn/ho-chi-minh/banh-canh-vit-cai-lay',
    'https://www.foody.vn/ho-chi-minh/tokyo-deli-phan-van-tri',
    'https://www.foody.vn/ho-chi-minh/sinh-to-ku-kun-no-trang-long',
    'https://www.foody.vn/ho-chi-minh/am-thuc-chay-huong-sen-vuon-lai',
    'https://www.foody.vn/ho-chi-minh/quan-banh-can-da-lat',
    'https://www.foody.vn/ho-chi-minh/banh-cuon-thao-29-huynh-man-dat',
    'https://www.foody.vn/ho-chi-minh/banh-trang-1989-banh-trang-tron-banh-trang-cuon',
    'https://www.foody.vn/ho-chi-minh/co-nha-banh-trang-muoi-nhuyen',
    'https://www.foody.vn/ho-chi-minh/tre-tron-coc-giang-hoang-tran-xuan-soan',
    'https://www.foody.vn/ho-chi-minh/com-ga-xoi-mo-142-ba-dinh',
    'https://www.foody.vn/ho-chi-minh/muc-280-muc-mot-nang-chien-nuoc-mam',
    'https://www.foody.vn/ho-chi-minh/tra-sua-binh-phu',
    'https://www.foody.vn/ho-chi-minh/tan-tong-loi-mi-sui-cao',
    'https://www.foody.vn/ho-chi-minh/tran-quang-ky-vit-quay-sot-tieu-den-luy-ban-bich',
    'https://www.foody.vn/ho-chi-minh/244-food-ca-ri-nhat-tao',
    'https://www.foody.vn/ho-chi-minh/lau-bo-ti-chuot',
    'https://www.foody.vn/ho-chi-minh/tokyo-deli-phan-xich-long',
    'https://www.foody.vn/ho-chi-minh/banh-bo-thot-not-cococake-lo-banh-minh-khai',
    'https://www.foody.vn/ho-chi-minh/chay-ngon-nhan-tam-com-chay',
    'https://www.foody.vn/ho-chi-minh/highlands-coffee-tra-ca-phe-banh-big-c-an-lac',
    'https://www.foody.vn/ho-chi-minh/sui-cao-ngo-gia-pham-van-chieu',
    'https://www.foody.vn/ho-chi-minh/mi-van-than-tuong-huyen',
    'https://www.foody.vn/ho-chi-minh/hu-tieu-ca-che-khiem',
    'https://www.foody.vn/ho-chi-minh/pho-dau-nam-ky-khoi-nghia',
    'https://www.foody.vn/ho-chi-minh/papa-lee-s-noodle-mi-hoanh-thanh-wonton-noodle-tran-phu',
    'https://www.foody.vn/ho-chi-minh/banh-canh-cua-co-lan-luy-ban-bich',
    'https://www.foody.vn/ho-chi-minh/pho-phu-vuong-su-van-hanh',
    'https://www.foody.vn/ho-chi-minh/koi-the-cafe-viet-nam-vincom-mega-mall-thao-dien',
    'https://www.foody.vn/ho-chi-minh/banh-xeo-cau-van-luy-ban-bich',
    'https://www.foody.vn/ho-chi-minh/quan-gia-he-com-suon-hu-tieu',
    'https://www.foody.vn/ho-chi-minh/pad-thai',
    'https://www.foody.vn/ho-chi-minh/a-bao-tra-dao-dam-dong-den',
    'https://www.foody.vn/ho-chi-minh/hu-tieu-nam-vang-nhan-quan-nguyen-thi-minh-khai',
    'https://www.foody.vn/ho-chi-minh/bun-bo-hue-lam-anh-nguyen-van-tang',
    'https://www.foody.vn/ho-chi-minh/dai-phat-su-van-hanh',
    'https://www.foody.vn/ho-chi-minh/tocotoco-bubble-tea-truong-son',
    'https://www.foody.vn/ho-chi-minh/sushi-buffet-kunimoto-truong-cong-dinh',
    'https://www.foody.vn/ho-chi-minh/sushi-buffet-kunimoto-143-thong-nhat',
    'https://www.foody.vn/ho-chi-minh/banh-duc-thich-quang-duc',
    'https://www.foody.vn/ho-chi-minh/che-hem-79-che-thap-cam-sam-bo-luong',
    'https://www.foody.vn/ho-chi-minh/bep-ba-thuy-ca-hoi-trung-long-dao-ngam-tuong-shop-online',
    'https://www.foody.vn/ho-chi-minh/pho-dat-nguyen-cao',
    'https://www.foody.vn/ho-chi-minh/banh-cuon-thu-hien-banh-cuon-van-kiep',
    'https://www.foody.vn/ho-chi-minh/dat-quan-chao-ech-singapore',
    'https://www.foody.vn/ho-chi-minh/co-loan-chao-suon-tan-hai',
    'https://www.foody.vn/ho-chi-minh/com-tam-tu-map',
    'https://www.foody.vn/ho-chi-minh/sapinkie-an-vat-4-teen',
    'https://www.foody.vn/ho-chi-minh/starbucks-coffee-panorama',
    'https://www.foody.vn/ho-chi-minh/quan-ba-ba-hu-tieu-my-banh-mi-bo-kho',
    'https://www.foody.vn/ho-chi-minh/quan-chao-211',
    'https://www.foody.vn/ho-chi-minh/vit-cay-so-1-singapore',
    'https://www.foody.vn/ho-chi-minh/phe-la-tra-ca-phe-dac-san-truong-dinh',
    'https://www.foody.vn/ho-chi-minh/banh-trang-tron-chu-vien',
    'https://www.foody.vn/ho-chi-minh/che-hien-khanh',
    'https://www.foody.vn/ho-chi-minh/the-gioi-steak-tan-son-nhi',
    'https://www.foody.vn/ho-chi-minh/thanh-dat-hu-tieu-nam-vang-hoang-hoa-tham',
    'https://www.foody.vn/ho-chi-minh/com-nha-40-hoa-hung',
    'https://www.foody.vn/ho-chi-minh/sup-cua-bong-mac-dinh-chi',
    'https://www.foody.vn/ho-chi-minh/morico-contemporary-japanese-lifestyle-diamond-island',
    'https://www.foody.vn/ho-chi-minh/ga-ran-kfc-trinh-van-can-nguyen-thai-hoc',
    'https://www.foody.vn/ho-chi-minh/xoi-heo-quay-ngoc-xoi-heo-quay-cmt8',
    'https://www.foody.vn/ho-chi-minh/banh-mi-81-huong-vi-pleiku-gia-lai-hoa-lan',
    'https://www.foody.vn/ho-chi-minh/tuy-duyen-banh-mi-chay',
    'http://foody.vn/ho-chi-minh/paris-baguette-cao-thang',
    'https://www.foody.vn/ho-chi-minh/pho-thai-son',
    'http://foody.vn/ho-chi-minh/bun-mam-nem-muoi-thoi-da-nang-phan-sao-nam',
    'https://www.foody.vn/ho-chi-minh/thanh-giong-express-banh-bot-loc-buon-me',
    'https://www.foody.vn/ho-chi-minh/che-thanh-tam-quan-5',
    'https://www.foody.vn/ho-chi-minh/sasin-mi-cay-7-cap-do-han-quoc-su-van-hanh',
    'https://www.foody.vn/ho-chi-minh/diem-quynh-ga-nuong-d4-binh-khanh',
    'https://www.foody.vn/ho-chi-minh/bun-dau-co-khan',
    'https://www.foody.vn/ho-chi-minh/quan-79-nem-nuong',
    'https://www.foody.vn/ho-chi-minh/xoi-sau-rieng-nguyen-thi-thap',
    'https://www.foody.vn/ho-chi-minh/pizza-hut-do-xuan-hop',
    'https://www.foody.vn/ho-chi-minh/suon-muoi-ot-phan-van-tri',
    'https://www.foody.vn/ho-chi-minh/hai-phong-chao-long-tiet-canh',
    'https://www.foody.vn/ho-chi-minh/ling-hu-tieu-sa-te',
    'https://www.foody.vn/ho-chi-minh/thien-an-banh-van-tran',
    'https://www.foody.vn/ho-chi-minh/ga-nuong-muoi-ot-bin-bo-289-ly-thai-to',
    'https://www.foody.vn/ho-chi-minh/banh-mi-hoang-lam',
    'https://www.foody.vn/ho-chi-minh/com-tam-phuc-loc-tho-an-duong-vuong',
    'https://www.foody.vn/ho-chi-minh/quan-chay-thanh-lac',
    'https://www.foody.vn/ho-chi-minh/banh-cuon-nam-viet',
    'https://www.foody.vn/ho-chi-minh/com-tam-40a-quoc-huong',
    'https://www.foody.vn/ho-chi-minh/viet-huong-ga-nuong-lu-pham-van-dong',
    'https://www.foody.vn/ho-chi-minh/thanh-thuy-am-thuc-mien-trung',
    'https://www.foody.vn/ho-chi-minh/beefsteak-titi',
    'https://www.foody.vn/ho-chi-minh/lang-vong-bun-cha-bun-dau-mam-tom',
    'https://www.foody.vn/ho-chi-minh/quan-hien-com-ga-phu-yen',
    'https://www.foody.vn/ho-chi-minh/bun-mam-144',
    'https://www.foody.vn/ho-chi-minh/quan-an-1976-mon-ngon-vi-bac-duong-so-20',
    'https://www.foody.vn/ho-chi-minh/bun-thit-nuong-com-tam-nui-xao-ma-bay',
    'https://www.foody.vn/ho-chi-minh/bot-chien-pham-the-hien',
    'https://www.foody.vn/ho-chi-minh/mi-chu-tac-ba-huyen-thanh-quang',
    'https://www.foody.vn/ho-chi-minh/vina-chuoi-co-bac',
    'https://www.foody.vn/ho-chi-minh/bun-dau-mac-van-khoa-phan-huy-ich',
    'https://www.foody.vn/ho-chi-minh/com-tam-phuong-ho-bieu-chanh',
    'https://www.foody.vn/ho-chi-minh/quan-che-thi-thi-ton-dan',
    'https://www.foody.vn/ho-chi-minh/vit-quay-vinh-phong',
    'https://www.foody.vn/ho-chi-minh/bun-bo-hai-le',
    'https://www.foody.vn/ho-chi-minh/banh-mi-xiu-mai-kho-nguyen-thi-minh-khai',
    'https://www.foody.vn/ho-chi-minh/z-coffee-tea',
    'https://www.foody.vn/ho-chi-minh/go-cafe-cao-dat',
    'https://www.foody.vn/ho-chi-minh/tra-sua-bobapop-hoang-van-thu',
    'https://www.foody.vn/ho-chi-minh/phuc-long-188-tran-hung-dao',
    'https://www.foody.vn/ho-chi-minh/highlands-coffee-luy-ban-bich',
    'https://www.foody.vn/ho-chi-minh/chut-chit-quan',
    'http://foody.vn/ho-chi-minh/sukiya-com-bo-ham-mi-ramen-ly-thai-to',
    'https://www.foody.vn/ho-chi-minh/bonchon-chicken-ngo-duc-ke',
    'https://www.foody.vn/ho-chi-minh/bun-cha-ca-nha-trang-hai-lua-cay-tram',
    'https://www.foody.vn/ho-chi-minh/khoai-tay-loc-xoay-140',
    'https://www.foody.vn/ho-chi-minh/osaka-com-tho-nhat-ban',
    'https://www.foody.vn/ho-chi-minh/banh-trang-tron-2-u-banh-trang-tron-top-mo-trung-cut-long-dao-1170-truong-sa',
    'https://www.foody.vn/ho-chi-minh/bun-bo-hue-14b',
    'https://www.foody.vn/ho-chi-minh/xoi-cay-hai-co',
    'https://www.foody.vn/ho-chi-minh/bun-bo-vy-da-xua',
    'https://www.foody.vn/ho-chi-minh/burger-king-thao-dien',
    'https://www.foody.vn/ho-chi-minh/tra-sua-pozaa-tea-hau-giang',
    'https://www.foody.vn/ho-chi-minh/tra-sua-126',
    'https://www.foody.vn/ho-chi-minh/bun-dau-a-vung-luong-huu-khanh',
    'https://www.foody.vn/ho-chi-minh/heo-quay-tay-bac-cho-ba-chieu',
    'https://www.foody.vn/ho-chi-minh/bep-me-bin-lau-69k-tra-sua-thien-phuoc',
    'https://www.foody.vn/ho-chi-minh/mew-coffee-no-trang-long',
    'https://www.foody.vn/ho-chi-minh/soup-cua-chi-bui-vo-thanh-trang',
    'https://www.foody.vn/ho-chi-minh/quan-dong-nhan-com-ba-ca-ton-that-thiep',
    'https://www.foody.vn/ho-chi-minh/thien-tran-nuoc-ep-trai-cay-nguyen-chat',
    'https://www.foody.vn/ho-chi-minh/banh-mi-ba-huynh-cong-quynh',
    'https://www.foody.vn/ho-chi-minh/passio-coffee-block-a-sunrise-cityview',
    'https://www.foody.vn/ho-chi-minh/mika-banh-mi-thit-bo-nuong-nguyen-thuong-hien',
    'https://www.foody.vn/ho-chi-minh/an-vat-quan-8-kho-bo-kho-ga-pham-the-hien.obn7gi',
    'https://www.foody.vn/ho-chi-minh/papas-chicken-ga-ran-han-quoc-nguyen-thien-thuat',
    'https://www.foody.vn/ho-chi-minh/ga-ran-chicken-plus-le-van-khuong',
    'https://www.foody.vn/ho-chi-minh/tra-chanh-hotdog-hang-xanh-323-dien-bien-phu',
    'https://www.foody.vn/ho-chi-minh/sushi-house',
    'https://www.foody.vn/ho-chi-minh/quan-chay-thoi-ke-le-hong-phong',
    'https://www.foody.vn/ho-chi-minh/com-tam-ba-chi-hai-com-tam-dang-van-ngu',
    'http://foody.vn/ho-chi-minh/chao-long-tan-hiep',
    'https://www.foody.vn/ho-chi-minh/tran-quang-ky',
    'https://www.foody.vn/ho-chi-minh/bun-dau-cau-go',
    'https://www.foody.vn/ho-chi-minh/king-bbq-vincom-center',
    'https://www.foody.vn/ho-chi-minh/viet-ngon-banh-cuon-nong-xoi-khuc',
    'https://www.foody.vn/ho-chi-minh/banh-duc-phan-dang-luu',
    'https://www.foody.vn/ho-chi-minh/banh-canh-ca-loc-xu-hue-nguyen-van-cong',
    'https://www.foody.vn/ho-chi-minh/katinat-bau-cat',
    'https://www.foody.vn/ho-chi-minh/bun-thit-nuong-kieu-bao-de-tham',
    'https://www.foody.vn/ho-chi-minh/bun-bo-yen-huong-giang',
    'https://www.foody.vn/ho-chi-minh/banh-canh-ca-loc-xu-hue-quang-trung',
    'https://www.foody.vn/ho-chi-minh/quan-ngu-binh-lam-van-ben',
    'https://www.foody.vn/ho-chi-minh/chuoi-nuong-vo-van-tan',
    'https://www.foody.vn/ho-chi-minh/banh-mi-que-26',
    'https://www.foody.vn/ho-chi-minh/banh-trang-chien-hung-phu',
    'https://www.foody.vn/ho-chi-minh/tung-tung-mon-an-han-quoc-tra-sua-truong-sa',
    'https://www.foody.vn/ho-chi-minh/kimbap-fc',
    'https://www.foody.vn/ho-chi-minh/busan-korean-food-mon-han-quoc-le-van-duyet',
    'https://www.foody.vn/ho-chi-minh/trieu-minh-hiep-tiem-banh-nguoi-hoa',
    'https://www.foody.vn/ho-chi-minh/lau-bo-123-lau-bo-ta-quang-buu.tfogpd',
    'https://www.foody.vn/ho-chi-minh/chao-long-ha-noi-nguyen-trai',
    'https://www.foody.vn/ho-chi-minh/lau-chao-long-co-beo-mon-ngon-gia-dinh-nguyen-thi-minh-khai.tgdpzh',
    'https://www.foody.vn/ho-chi-minh/sasin-mi-cay-7-cap-do-han-quoc-hoa-binh',
    'https://www.foody.vn/ho-chi-minh/che-beans-the-gioi-che-dau',
    'https://www.foody.vn/ho-chi-minh/bun-bo-hue-chu-ha',
    'https://www.foody.vn/ho-chi-minh/ga-met-duong-c18',
    'https://www.foody.vn/ho-chi-minh/com-chay-dieu-nhi-phan-huy-ich',
    'https://www.foody.vn/ho-chi-minh/cajun-tom-379',
    'https://www.foody.vn/ho-chi-minh/lotteria-572a-ba-thang-hai',
    'https://www.foody.vn/ho-chi-minh/khoi-pizza-pizza-tran-chau-duong-den-nguyen-huu-canh',
    'https://www.foody.vn/ho-chi-minh/quan-chay-tue-nghiem-trai-am-thuc-chay-nguyen-huu-canh',
    'https://www.foody.vn/ho-chi-minh/bun-dau-quynh-hong',
    'https://www.foody.vn/ho-chi-minh/banh-canh-cha-ca-phu-quoc-68-huynh-van-banh',
    'https://www.foody.vn/ho-chi-minh/com-ga-hai-nam-au-co',
    'https://www.foody.vn/ho-chi-minh/tiem-lau-bo-thanh-dat-52-duong-79',
    'https://www.foody.vn/ho-chi-minh/tl-sushi-sashimi-phan-van-han',
    'https://www.foody.vn/ho-chi-minh/richii-com-ga-sot-duong-so-6',
    'https://www.foody.vn/ho-chi-minh/nhat-thien-moc-soup-vi-ca-bangkok',
    'https://www.foody.vn/ho-chi-minh/com-tam-cay-khe-3-nguyen-van-khoi',
    'https://www.foody.vn/ho-chi-minh/bun-bo-hue-hanh',
    'https://www.foody.vn/ho-chi-minh/yen-banh-cuon-cha-muc-ha-long-cha-muc-ha-long-no-trang-long',
    'https://www.foody.vn/ho-chi-minh/ca-vien-nam-sao-duong-so-17',
    'https://www.foody.vn/ho-chi-minh/quan-bui-authentic-vietnamese-cuisine',
    'https://www.foody.vn/ho-chi-minh/ca-vien-chien-luong-huu-khanh',
    'https://www.foody.vn/ho-chi-minh/ba-hoang-hu-tieu-nam-vang',
    'https://www.foody.vn/ho-chi-minh/banh-xeo-ngoc-son',
    'https://www.foody.vn/ho-chi-minh/gac-lung-mi-tron-an-vat-le-van-sy.6hvwh6',
    'https://www.foody.vn/ho-chi-minh/nghien-store-banh-trang-an-vat-ly-thuong-kiet',
    'https://www.foody.vn/ho-chi-minh/cong-caphe-cho-lon',
    'https://www.foody.vn/ho-chi-minh/the-gioi-com-tam-bun-thit-nuong-banh-mi-thit-nuong-phan-van-han',
    'https://www.foody.vn/ho-chi-minh/chao-ech-singapore-ha-vu',
    'https://www.foody.vn/ho-chi-minh/bun-quay-tam-quan-phan-van-tri',
    'https://www.foody.vn/ho-chi-minh/quan-trung-bun-bo-mi-quang-mon-ngon-mien-trung',
    'https://www.foody.vn/ho-chi-minh/co-hue-bun-dau-mam-tom-banh-tom-ho-tay',
    'https://www.foody.vn/ho-chi-minh/chao-suon-hai-ba-trung-tan-dinh',
    'https://www.foody.vn/ho-chi-minh/chao-suon-co-be',
    'https://www.foody.vn/ho-chi-minh/com-ga-chao-mien-goi-ga-ta-tuong-vy-thach-lam',
    'https://www.foody.vn/ho-chi-minh/homies-tra-sua-pho-mai-tuoi-dai-loan',
    'https://www.foody.vn/ho-chi-minh/mi-quang-an-la-nho-tran-quoc-thao',
    'https://www.foody.vn/ho-chi-minh/som-thai-le-thanh-ton',
    'https://www.foody.vn/ho-chi-minh/quan-tu-loan-bun-thit-nuong-nem-nuong-goi-cuon',
    'https://www.foody.vn/ho-chi-minh/nem-lui-co-dao',
    'https://www.foody.vn/ho-chi-minh/highlands-coffee-kdc-trung-son',
    'https://www.foody.vn/ho-chi-minh/starbucks-coffee-golden-mansion',
    'https://www.foody.vn/ho-chi-minh/yen-cua-goi-cuon-cua-bien-hanh-yen',
    'https://www.foody.vn/ho-chi-minh/7-eleven-cua-hang-tien-loi-hau-giang',
    'https://www.foody.vn/ho-chi-minh/com-chien-nui-xao-bo-cong-quynh',
    'https://www.foody.vn/ho-chi-minh/banh-trang-muoi-tom-tue-tinh',
    'https://www.foody.vn/ho-chi-minh/ga-ran-han-quoc-k-jeju-chicken',
    'https://www.foody.vn/ho-chi-minh/tra-sua-maycha-ton-dan',
    'https://www.foody.vn/ho-chi-minh/lau-bo-tien-duong',
    'https://www.foody.vn/ho-chi-minh/tra-sua-gia-nghi',
    'https://www.foody.vn/ho-chi-minh/ca-vien-chien-nuoc-mam-kem-kep-singapore-tam-dao',
    'https://www.foody.vn/ho-chi-minh/bun-bo-hue-62',
    'https://www.foody.vn/ho-chi-minh/ga-ran-burger-lotteria-bach-dang',
    'https://www.foody.vn/ho-chi-minh/ech-xanh',
    'https://www.foody.vn/ho-chi-minh/pho-ga-chao-ga-ngo-gia-tu',
    'http://foody.vn/ho-chi-minh/banh-mi-cha-ca-bui-thi-xuan',
    'https://www.foody.vn/ho-chi-minh/kfc-nguyen-anh-thu',
    'https://www.foody.vn/ho-chi-minh/jino-banh-trang-thanh-mai-tra-sua',
    'https://www.foody.vn/ho-chi-minh/thai-express-mon-thai-sc-vivocity',
    'https://www.foody.vn/ho-chi-minh/co-ha-bun-nem-nuong-cha-gio',
    'https://www.foody.vn/ho-chi-minh/ararat-armenian-cuisine',
    'https://www.foody.vn/ho-chi-minh/papas-chicken-ga-ran-han-quoc-hoang-van-thu',
    'https://www.foody.vn/ho-chi-minh/xoi-lang-nghe-chuan-vi-bac-nguyen-thi-minh-khai',
    'https://www.foody.vn/ho-chi-minh/com-ga-xoi-mo-hong-ky',
    'https://www.foody.vn/ho-chi-minh/co-chu-nho-tokbokki-my-cay-banh-gao-han-quoc-nguyen-canh-chan',
    'https://www.foody.vn/ho-chi-minh/sau-rieng-minh-chau-com-sau-rieng-sau-rieng-trai-nuoc-mia',
    'https://www.foody.vn/ho-chi-minh/bun-thai-va-bun-mam-dung-chien-luoc',
    'https://www.foody.vn/ho-chi-minh/banh-cuon-banh-uot-phu-lam',
    'https://www.foody.vn/ho-chi-minh/pho-bo-phu-gia',
    'https://www.foody.vn/ho-chi-minh/banh-mi-pha-lau-xa-xiu-27a-nguyen-van-dau',
    'https://www.foody.vn/ho-chi-minh/co-thuy-xoi-nep-than',
    'https://www.foody.vn/ho-chi-minh/xien-que-ngon',
    'https://www.foody.vn/ho-chi-minh/bun-thit-nuong-kieu-bao-le-trong-tan',
    'https://www.foody.vn/ho-chi-minh/quan-bun-ngan-pho-co',
    'https://www.foody.vn/ho-chi-minh/di-bay-bun-mam-nem-da-nang',
    'https://www.foody.vn/ho-chi-minh/terra-coffee-pham-ngoc-thach',
    'https://www.foody.vn/ho-chi-minh/banh-xeo-co-nguyet-77-le-duc-tho',
    'https://www.foody.vn/ho-chi-minh/cuong-ky-mi-gia-ha-ton-quyen',
    'https://www.foody.vn/ho-chi-minh/domino-s-pizza-huynh-tan-phat-2',
    'https://www.foody.vn/ho-chi-minh/oc-chu-lun',
    'https://www.foody.vn/ho-chi-minh/mien-ga-son-nga',
    'https://www.foody.vn/ho-chi-minh/vi-que-com-tam-cu-lao-1',
    'https://www.foody.vn/ho-chi-minh/bun-mam-444',
    'https://www.foody.vn/ho-chi-minh/linh-trang-anh-pho-ga-bun-thang-pho-co-ha-noi-duong-79',
    'https://www.foody.vn/ho-chi-minh/banh-xeo-mien-trung-198-le-van-luong',
    'https://www.foody.vn/ho-chi-minh/com-chay-kho-quet-phu-lam-2',
    'https://www.foody.vn/ho-chi-minh/mi-quang-vinh',
    'https://www.foody.vn/ho-chi-minh/com-ga-thuong-hai-am-thuc-trung-hoa',
    'https://www.foody.vn/ho-chi-minh/com-ga-hai-nam-binh-gia',
    'https://www.foody.vn/ho-chi-minh/banh-trang-kim-ngoc-nguyen-trai',
    'https://www.foody.vn/ho-chi-minh/kfc-hau-giang',
    'https://www.foody.vn/ho-chi-minh/kfc-nguyen-van-giai',
    'https://www.foody.vn/ho-chi-minh/a-dat-banh-xeo-rau-rung',
    'https://www.foody.vn/ho-chi-minh/canh-bun-me-toi',
    'https://www.foody.vn/ho-chi-minh/mi-y-me-ri',
    'https://www.foody.vn/ho-chi-minh/highlands-coffee-tra-ca-phe-banh-55-vuon-lai',
    'https://www.foody.vn/ho-chi-minh/banh-canh-cua-14-tran-binh-trong',
    'https://www.foody.vn/ho-chi-minh/banh-mi-thit-nuong-co-huong-banh-mi-xoi-bun-nguyen-van-nghi',
    'https://www.foody.vn/ho-chi-minh/sup-cua-le-dinh-can',
    'https://www.foody.vn/ho-chi-minh/bun-thit-nuong-kieu-bao-quang-trung',
    'https://www.foody.vn/ho-chi-minh/vit-quay-toan-thanh',
    'https://www.foody.vn/ho-chi-minh/chao-long-144',
    'https://www.foody.vn/ho-chi-minh/cheese-coffee-cong-hoa',
    'https://www.foody.vn/ho-chi-minh/hoang-ty-group-dac-san-trang-bang-mac-dinh-chi',
    'https://www.foody.vn/ho-chi-minh/lau-bo-nha-go-phan-van-tri',
    'https://www.foody.vn/ho-chi-minh/ga-ran-burger-lotteria-sieu-thi-mega-market-an-phu',
    'https://www.foody.vn/ho-chi-minh/ga-ran-burger-lotteria-nguyen-van-qua',
    'https://www.foody.vn/ho-chi-minh/com-tam-hung-hem-194-vo-van-tan',
    'https://www.foody.vn/ho-chi-minh/ho-lo-coffee',
    'https://www.foody.vn/ho-chi-minh/ca-loc-nuong-mia-tran-quang-co',
    'https://www.foody.vn/ho-chi-minh/thi-ca-loc-nuong',
    'https://www.foody.vn/ho-chi-minh/com-tam-an-duong-vuong',
    'https://www.foody.vn/ho-chi-minh/pho-hoang-nguyen-tri-phuong',
    'https://www.foody.vn/ho-chi-minh/quan-o-tu-bun-bo-hue',
    'https://www.foody.vn/ho-chi-minh/hu-tieu-mi-ga-nguyen-kim-nhat-tao',
    'https://www.foody.vn/ho-chi-minh/bap-xao-ho-con-rua',
    'https://www.foody.vn/ho-chi-minh/quan-phuc-loc-tho-com-tam',
    'https://www.foody.vn/ho-chi-minh/the-pizza-company-phan-van-tri',
    'https://www.foody.vn/ho-chi-minh/texas-chicken-phan-van-tri',
    'https://www.foody.vn/ho-chi-minh/phat-ky-com-chien-mi-xao-hu-tieu-xao-hoang-dieu-2',
    'https://www.foody.vn/ho-chi-minh/som-tum-thai-nguyen-tri-phuong',
    'https://www.foody.vn/ho-chi-minh/quan-76-dac-san-mien-trung',
    'https://www.foody.vn/ho-chi-minh/ga-ran-burger-lotteria-tttm-crescent-mall',
    'https://www.foody.vn/ho-chi-minh/xuc-xich-duc-anh-ba-tay-duong-ba-trac',
    'https://www.foody.vn/ho-chi-minh/bun-moc-hung-map-nguyen-van-qua',
    'https://www.foody.vn/ho-chi-minh/banh-canh-ghe-bay-lien',
    'https://www.foody.vn/ho-chi-minh/chao-dinh-duong-wow-duong-so-19',
    'https://www.foody.vn/ho-chi-minh/vuon-oc-wongnai',
    'https://www.foody.vn/ho-chi-minh/buddha-chay',
    'https://www.foody.vn/ho-chi-minh/che-to-yen-vitamin-shop-online',
    'https://www.foody.vn/ho-chi-minh/yen-sao-thuong-hang-kimberley-yen-chung-hu-yen-cao-cap',
    'https://www.foody.vn/ho-chi-minh/fox-tea-tra-sua-an-vat-duong-so-8',
    'https://www.foody.vn/ho-chi-minh/sukiya-com-bo-ham-mi-ramen',
    'https://www.foody.vn/ho-chi-minh/chao-tre-em-chao-dinh-duong-uno-ly-thai-to',
    'https://www.foody.vn/ho-chi-minh/gio-cha-nghia-nguyen-dinh-chieu',
    'https://www.foody.vn/ho-chi-minh/co-hoa-com-tam-bun-thit-nuong',
    'https://www.foody.vn/ho-chi-minh/tuyetfood-banh-mi-bo-kho',
    'https://www.foody.vn/ho-chi-minh/thuy-linh-banh-da-cua-hai-phong',
    'https://www.foody.vn/ho-chi-minh/quan-ha-banh-cuon-nong-banh-duc',
    'https://www.foody.vn/ho-chi-minh/che-cot-dien-che-nguoi-hoa-khu-den-nam-ngon',
    'https://www.foody.vn/ho-chi-minh/ga-ran-burger-lotteria-tuy-ly-vuong-coopmart',
    'https://www.foody.vn/ho-chi-minh/the-gioi-tau-hu',
    'https://www.foody.vn/ho-chi-minh/quan-bun-thit-nuong-cay-man',
    'https://www.foody.vn/ho-chi-minh/kfc-pandora-city',
    'https://www.foody.vn/ho-chi-minh/texas-chicken-aeon-mall-tan-phu',
    'https://www.foody.vn/ho-chi-minh/quan-minh-ky-dimsum-binh-dan',
    'https://www.foody.vn/ho-chi-minh/the-coffee-house-313-nguyen-thi-thap',
    'https://www.foody.vn/ho-chi-minh/che-ba-mau-xoi-xiem',
    'https://www.foody.vn/ho-chi-minh/ga-nuong-o-o-o-ga-nuong-nhieu-vi-le-quang-dinh',
    'https://www.foody.vn/ho-chi-minh/co-tin-bun-thit-nuong-com-tam',
    'https://www.foody.vn/ho-chi-minh/nha-hang-hai-san-phi-pho-bien',
    'https://www.foody.vn/ho-chi-minh/koi-the-phan-van-tri',
    'https://www.foody.vn/ho-chi-minh/chicc-chicc-ga-ran-han-quoc',
    'https://www.foody.vn/ho-chi-minh/ba-hao-quan-chao-suon',
    'https://www.foody.vn/ho-chi-minh/hoa-hong-banh-mi-xoi-banh-bao',
    'https://www.foody.vn/ho-chi-minh/bachutha-banh-chuoi-thai-hoang-sa',
    'https://www.foody.vn/ho-chi-minh/3-rau-ga-ran-pizza-tra-sua-ton-dan',
    'https://www.foody.vn/ho-chi-minh/mr-nem-nem-nuong-nha-trang-bun-thit-nuong-cay-tram',
    'https://www.foody.vn/ho-chi-minh/mr-nem-nem-nuong-nha-trang-bun-thit-nuong-ho-dac-di',
    'https://www.foody.vn/ho-chi-minh/koi-the-bau-cat',
    'https://www.foody.vn/ho-chi-minh/com-tam-sa-bi-chuong-tran-binh-trong',
    'https://www.foody.vn/ho-chi-minh/thanh-thao-ly-tu-trong',
    'https://www.foody.vn/ho-chi-minh/ga-ran-burger-mcdonald-s-vincom-masteri',
    'https://www.foody.vn/ho-chi-minh/quan-trang-banh-canh-ca-loc',
    'https://www.foody.vn/ho-chi-minh/coconino-tea-cheese-phan-van-tri',
    'https://www.foody.vn/ho-chi-minh/mi-tuong-den-hakenfu',
    'https://www.foody.vn/ho-chi-minh/he-he-mi-tron-top-mo-da-ga-bo-mam',
    'https://www.foody.vn/ho-chi-minh/che-3-2-hem-992-diem-hong',
    'https://www.foody.vn/ho-chi-minh/banh-mi-kebab-tho-nhi-ky',
    'https://www.foody.vn/ho-chi-minh/banh-mi-chi-hong',
    'https://www.foody.vn/ho-chi-minh/com-tam-tam-nga-689-le-van-luong',
    'https://www.foody.vn/ho-chi-minh/che-buoi-vinh-long-ngo-gia-tu',
    'https://www.foody.vn/ho-chi-minh/minh-com-tam-dem',
    'https://www.foody.vn/ho-chi-minh/tiem-tra-ba-xa-nguyen-kiem',
    'https://www.foody.vn/ho-chi-minh/chu-sai-hu-tiu-mi',
    'https://www.foody.vn/ho-chi-minh/tuyen-ky-banh-mi-heo-quay',
    'https://www.foody.vn/ho-chi-minh/com-tam-khuong-com-tam-suon-cong-hoa-binh',
    'https://www.foody.vn/ho-chi-minh/pho-dung-nam-dinh-cao-thang',
    'http://foody.vn/ho-chi-minh/com-ga-chao-goi-ga-ta-tuong-vy-xo-viet-nghe-tinh',
    'https://www.foody.vn/ho-chi-minh/ga-ran-burger-lotteria-nguyen-tri-phuong',
    'https://www.foody.vn/ho-chi-minh/com-chay-dieu-vy-au-co',
    'https://www.foody.vn/ho-chi-minh/gong-cha-tra-ca-phe-nguyen-son',
    'https://www.foody.vn/ho-chi-minh/hoa-dong-quan-bun-cha',
    'https://www.foody.vn/ho-chi-minh/quan-cay-sung-com-tam-bun-rieu-canh-bun',
    'https://www.foody.vn/ho-chi-minh/clumsy-chuan-gu-dam-vi-nguyen-trai',
    'https://www.foody.vn/ho-chi-minh/banh-mi-madame-mun',
    'https://www.foody.vn/ho-chi-minh/co-ba-che-com-tam-long-xuyen-nguyen-thai-binh',
    'https://www.foody.vn/ho-chi-minh/bap-nuong-mo-hanh-hem-174-nguyen-thien-thuat',
    'https://www.foody.vn/ho-chi-minh/gold-chicken-quan-1',
    'https://www.foody.vn/ho-chi-minh/foodfest-bento-delichi-com-ga-xoi-mam-toi-xoi-mo-quan-1-nowfoodxcoca-cola',
    'https://www.foody.vn/ho-chi-minh/bun-mam-nem-chi-linh-quan-1-15-le-thanh-ton',
    'https://www.foody.vn/ho-chi-minh/ga-nuong-lau-ga-de-nhat-226-dinh-tien-hoang-quan-1',
    'https://www.foody.vn/ho-chi-minh/mi-tron-xp-deli-quan-1',
    'https://www.foody.vn/ho-chi-minh/oc-nuot-cac-mon-ngon-tu-oc-tai-quan-1',
    'https://www.foody.vn/ho-chi-minh/com-hai-di-com-van-phong-quan-1',
    'https://www.foody.vn/ho-chi-minh/bep-khoi-huong-nam-bun-thai-tomyum-quan-1',
    'https://www.foody.vn/ho-chi-minh/bun-xa-xiu-ong-chu-nho-quan-1',
    'https://www.foody.vn/ho-chi-minh/nha-cua-bung-doi-com-trua-quan-1',
    'https://www.foody.vn/ho-chi-minh/ut-phuong-com-ga-chay-toi-35k-quan-1',
    'https://www.foody.vn/ho-chi-minh/banh-mi-ban-me-cn-quan-1',
    'https://www.foody.vn/ho-chi-minh/com-tam-sai-gon-quan-1',
    'https://www.foody.vn/ho-chi-minh/korea-no-1-mala-sorimmara-quan-1',
    'https://www.foody.vn/ho-chi-minh/q-cafe-studio',
    'https://www.foody.vn/ho-chi-minh/tous-les-jours-vincom-center--',
    'https://www.foody.vn/ho-chi-minh/manwah-taiwanese-hotpot-an-duong-vuong',
    'https://www.foody.vn/ho-chi-minh/tra-sua-tien-huong',
    'https://www.foody.vn/ho-chi-minh/hoa-tuc',
    'https://www.foody.vn/ho-chi-minh/quan-com-ba-map-138-com-trua-vo-van-tan',
    'https://www.foody.vn/ho-chi-minh/katinat-cong-hoa',
    'https://www.foody.vn/ho-chi-minh/3-rau-ga-ran-pizza-tra-sua-le-trong-tan',
    'https://www.foody.vn/ho-chi-minh/bun-thit-nuong-kieu-bao-to-hien-thanh',
    'https://www.foody.vn/ho-chi-minh/quan-go-banh-xeo-rau-rung-cu-hu-dua',
    'https://www.foody.vn/ho-chi-minh/com-tam-ba-ghien-khong-chi-nhanh-84-dang-van-ngu',
    'https://www.foody.vn/ho-chi-minh/che-hoa-tran-cao-van',
    'https://www.foody.vn/ho-chi-minh/dut-quan-com-van-phong-nguyen-huu-hao',
    'https://www.foody.vn/ho-chi-minh/highlands-coffee-tra-ca-phe-banh-phan-xich-long',
    'https://www.foody.vn/ho-chi-minh/delicious-sake-su-van-hanh',
    'https://www.foody.vn/ho-chi-minh/ganesh-am-thuc-an-do-duong-so-6',
    'https://www.foody.vn/ho-chi-minh/bo-bia-ly',
    'https://www.foody.vn/ho-chi-minh/quan-chay-thuan-duc',
    'https://www.foody.vn/ho-chi-minh/hanh-dung-nha-hang-gia-dinh',
    'https://www.foody.vn/ho-chi-minh/nha-hang-chay-kim-sa-binh-thanh',
    'https://www.foody.vn/ho-chi-minh/quan-chay-quan-am',
    'https://www.foody.vn/ho-chi-minh/com-chien-binh-nga',
    'https://www.foody.vn/ho-chi-minh/vit-quay-heo-quay-6-phi-nguyen-van-tang',
    'https://www.foody.vn/ho-chi-minh/duy-ga',
    'https://www.foody.vn/ho-chi-minh/ga-ran-burger-mcdonald-s-au-co',
    'https://www.foody.vn/ho-chi-minh/co-nam-cut-chien-bo',
    'https://www.foody.vn/ho-chi-minh/hanuri-quan-an-han-quoc-xo-viet-nghe-tinh'
]



results = []
for idx, url in enumerate(urls, start=400):
    data = crawl_data(url, idx)
    for comment in data:
        results.append(comment)

# Xuất JSON
import json
with open("restaurants_reviews.json", "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=4)


⛔ Không tìm thấy review nào cho https://www.foody.vn/ho-chi-minh/hoa-hong-banh-mi-xoi-banh-bao
